In [ ]:
!pip install torch torchvision opencv-python transformers

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    return frame_count, output_folder


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

# Transformation pipeline
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def segment_object(frame):
    input_image = Image.open(frame)
    input_tensor = preprocess(input_image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]

    return torch.argmax(output, dim=0).byte().cpu().numpy()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:01<00:00, 165MB/s]


In [ ]:
import numpy as np

def apply_background_change(frame_path, mask, background_color):
    image = cv2.imread(frame_path)
    background_mask = (mask == 0)  # Assuming background label is 0
    color = np.array(background_color, dtype=np.uint8)

    image[background_mask] = color
    return image

def color_from_prompt(prompt):
    # Example function to map text color prompt to RGB values
    colors = {
        "red": [0, 0, 255],
        "green": [0, 255, 0],
        "blue": [255, 0, 0],
        "yellow": [0, 255, 255],
    }
    for color in colors:
        if color in prompt:
            return colors[color]
    return [255, 255, 255]  # Default to white


In [ ]:
def process_video(input_folder, frame_count, output_video, prompt):
    background_color = color_from_prompt(prompt)

    height, width, layers = cv2.imread(os.path.join(input_folder, "frame_0.jpg")).shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, 20.0, (width, height))

    for i in range(frame_count):
        frame_path = os.path.join(input_folder, f"frame_{i}.jpg")
        mask = segment_object(frame_path)
        edited_frame = apply_background_change(frame_path, mask, background_color)

        video_writer.write(edited_frame)

    video_writer.release()


In [ ]:
def change_background_in_video(video_path, prompt, output_video="output.mp4"):
    # Step 1: Extract frames
    frame_count, frame_folder = extract_frames(video_path, "frames")

    # Step 2: Process the frames and apply background change
    process_video(frame_folder, frame_count, output_video, prompt)

# Example usage
video_path = "parkour.mp4"
prompt = "change background to green"
output_video = "edited_video.mp4"
change_background_in_video(video_path, prompt, output_video)
